In [2]:
#imports

import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [3]:
df = pd.read_csv("Project DataSet 4.25.23.csv")

In [4]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,Marketing Flags_content,available size options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,1284462.0,Acqua Di Parma,Cologne,Colonia Essenza,1.7 oz/ 50 mL,5.0,13.0,719.0,120.0,120.0,...,TRUE,online only,- 1.7 oz/ 50 mL Eau de Cologne Spray - 3.4 o...,The newest Eau de Cologne from Acqua di Parma ...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Hy...,1.0,0.0,0.0,0.0
1,2221588.0,Acqua Di Parma,Body Mist & Hair Mist,Peonia Nobile Hair Mist,1.7oz/50mL,4.0,5.0,800.0,58.0,58.0,...,TRUE,online only,- 1.7oz/50mL Hair Mist - 0.70oz/20mL Eau de...,A new Le Nobili beauty ritual leaves your hair...,Suggested Usage:-Spray over your hair to leave...,Alcohol Denat.- Water- Fragrance- Polyquaterni...,1.0,0.0,0.0,0.0
2,2222917.0,Acqua Di Parma,Body Lotions & Body Oils,Fico di Amalfi Spray Body Lotion,5 oz/150 mL,3.0,2.0,420.0,55.0,55.0,...,TRUE,online only,no options,Fragrance Family: FloralScent Type: Fruity Flo...,Suggested Usage:-Spray generously onto clean s...,-Almond Oil- Olive Oil- and Grapeseed Oil: Hy...,1.0,0.0,0.0,0.0
3,967224.0,Acqua Di Parma,Body Sprays & Deodorant,Colonia Alcohol-Free Deodorant,2.7 oz/ 77 g,3.5,7.0,400.0,50.0,50.0,...,TRUE,online only,no options,What it is:A deodorant stick that wears comfor...,no instructions,Aqua (Water)- Butylene Glycol- Glycerin- Sodiu...,1.0,0.0,0.0,0.0
4,1059500.0,Acqua Di Parma,Cologne,Colonia Intensa,1.7 oz/ 50 mL,4.0,20.0,584.0,120.0,120.0,...,TRUE,online only,- 1.7 oz/ 50 mL Eau de Cologne Spray,Colonia Intensa- the first Eau de Cologne pure...,no instructions,After Shave Balm:-Aloe Vera Gel: Soothes irrit...,1.0,0.0,0.0,0.0


In [5]:
df.shape

(5872, 21)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5872 entries, 0 to 5871
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5866 non-null   float64
 1   brand                    5866 non-null   object 
 2   category                 5866 non-null   object 
 3   name                     5866 non-null   object 
 4   size                     5866 non-null   object 
 5   rating                   5866 non-null   float64
 6   number_of_reviews        5866 non-null   float64
 7   love                     5866 non-null   float64
 8   price                    5866 non-null   float64
 9   value_price              5866 non-null   float64
 10  URL                      5869 non-null   object 
 11  MarketingFlags           5869 non-null   object 
 12  Marketing Flags_content  5869 non-null   object 
 13  available size options   5869 non-null   object 
 14  details                 

## Notice the RangeIndex in the cell above. Each column should have 5871 non-null values due to the 5871 records. Each column currently has 5866, which means 5 records have no data. After looking at the data, those 5 records are all from the previous preprocesing in Excel, with empty cells.

In [7]:
df = df.dropna(axis=0,how='any')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5866 entries, 0 to 5865
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5866 non-null   float64
 1   brand                    5866 non-null   object 
 2   category                 5866 non-null   object 
 3   name                     5866 non-null   object 
 4   size                     5866 non-null   object 
 5   rating                   5866 non-null   float64
 6   number_of_reviews        5866 non-null   float64
 7   love                     5866 non-null   float64
 8   price                    5866 non-null   float64
 9   value_price              5866 non-null   float64
 10  URL                      5866 non-null   object 
 11  MarketingFlags           5866 non-null   object 
 12  Marketing Flags_content  5866 non-null   object 
 13  available size options   5866 non-null   object 
 14  details                 

In [9]:
#checking to see if any columns are missing values
df.columns[df.isna().any()].tolist()

[]

In [10]:
#filtering products based on the specified ingredients
ingredients = ["avobenzone", "ecamsule", "oxybenzone", "octocrylene", "zinc oxide", "titanium dioxide", "octinoxate"]
df_filtered = df[df["ingredients"].str.contains('|'.join(ingredients), case=False)]

In [11]:
#using TF-IDF to vectorize the ingredients of the products
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_filtered["ingredients"])

In [12]:
#from sklearn.metrics.pairwise import cosine_similarity
#cosine_similarities = cosine_similarity(tfidf_matrix)

In [40]:
#using KNN as our model
k = 11
neigh = NearestNeighbors(n_neighbors=k, metric='cosine')
neigh.fit(tfidf_matrix)
top_k_indexes = neigh.kneighbors(tfidf_matrix, return_distance=False)

In [41]:
#define the target products 
target_products = df_filtered[df_filtered["name"].str.contains("sunscreen|spf", case=False)]
target_products = target_products.sort_values(by=['rating', 'number_of_reviews'], ascending=False)

In [44]:
# iterate over the target products
for i, product in target_products.iterrows():
    print('\nProduct Name:', product['name'])
    print('Rating:', product['rating'])
    #print('Number of Reviews:', product['number_of_reviews'])
    
    #get the index of the current product in the cosine similarities matrix
    index = df_filtered.index.get_loc(i)
    
    #Get the top k similar products
    top_k_indexes = neigh.kneighbors(tfidf_matrix[index], n_neighbors=k+1, return_distance=False)
    
    #remove the index of the current product from the list of similar products
    top_k_indexes = top_k_indexes[0][1:]
    
    #get the recommended products based on the indexes
    recommended_products = df_filtered.iloc[top_k_indexes]
    
    #sort the recommended products by rating and number of reviews
    recommended_products = recommended_products.sort_values(by=['rating', 'number_of_reviews'], ascending=False)
    
    #print 5 recommended products
    print('Recommended products:')
    for index, recommended_product in recommended_products.iloc[0:5].iterrows():
        print('- ' + recommended_product['name'], '\n\tRating: ', recommended_product['rating'],'\n\tNumber of Reviews: ', recommended_product['number_of_reviews'])


Product Name: Every Sun Day™ Mineral Sunscreen SPF 50+
Rating: 5.0
Recommended products:
- Modern Friction™ Nature’s Gentle Dermabrasion 
	Rating:  4.5 
	Number of Reviews:  437.0
- Ginger Souffle™ Whipped Body Cream 
	Rating:  4.5 
	Number of Reviews:  86.0
- Glow One Step Further 
	Rating:  4.5 
	Number of Reviews:  4.0
- Sephora Heroes Kit 
	Rating:  4.5 
	Number of Reviews:  4.0
- Premium Beauty Balm SPF 45 
	Rating:  4.0 
	Number of Reviews:  2000.0

Product Name: Invisible Physical Defense Sunscreen SPF 30
Rating: 5.0
Recommended products:
- Pep-Start Daily UV Protector Broad Spectrum SPF 50 
	Rating:  4.5 
	Number of Reviews:  286.0
- City Skin Age Defense Broad Spectrum SPF 50 PA++++ 
	Rating:  4.5 
	Number of Reviews:  236.0
- The Etherealist Foundation 
	Rating:  4.5 
	Number of Reviews:  195.0
- Even Better™ Glow Light Reflecting Makeup Broad Spectrum SPF 15 Foundation 
	Rating:  4.5 
	Number of Reviews:  124.0
- Ultra HD Soft Light Liquid Highlighter 
	Rating:  4.5 
	Numbe

Recommended products:
- Your Skin But Better Makeup Primer+ 
	Rating:  4.5 
	Number of Reviews:  372.0
- Teint Couture Blurring Foundation Balm Broad Spectrum 15 
	Rating:  4.5 
	Number of Reviews:  216.0
- Diorskin Forever & Ever Wear Makeup Primer SPF 20 
	Rating:  4.5 
	Number of Reviews:  95.0
- Matte Velvet Skin Full Coverage Foundation 
	Rating:  4.0 
	Number of Reviews:  1000.0
- Teint Couture Everwear Foundation 
	Rating:  4.0 
	Number of Reviews:  109.0

Product Name: Diorskin Forever & Ever Wear Makeup Primer SPF 20
Rating: 4.5
Recommended products:
- Synchro Skin Self-Refreshing Custom Finish Powder Foundation 
	Rating:  4.5 
	Number of Reviews:  1000.0
- Dior Forever 24h* Wear High Perfection Skin-Caring Matte Foundation 
	Rating:  4.5 
	Number of Reviews:  989.0
- Teint Couture Blurring Foundation Balm Broad Spectrum 15 
	Rating:  4.5 
	Number of Reviews:  216.0
- Ultra HD Soft Light Liquid Highlighter 
	Rating:  4.5 
	Number of Reviews:  52.0
- Superbalanced Silk Makeup B

Recommended products:
- Even Better™ Glow Light Reflecting Makeup Broad Spectrum SPF 15 Foundation 
	Rating:  4.5 
	Number of Reviews:  124.0
- DayWear Multi-Protection Anti-Oxidant Sheer Tint Release Moisturizer Broad Spectrum SPF 15 - Normal/ Combination Skin 
	Rating:  4.5 
	Number of Reviews:  20.0
- Age Defense BB Cream Broad Spectrum SPF 30 
	Rating:  4.0 
	Number of Reviews:  756.0
- Pore Refining Solutions Correcting Serum 
	Rating:  4.0 
	Number of Reviews:  404.0
- Primer Plus Radiance SPF 35 
	Rating:  4.0 
	Number of Reviews:  66.0

Product Name: Super Serum Skin Tint SPF 40 Foundation
Rating: 4.0
Recommended products:
- Original Skin™ Retexturizing Mask with Rose Clay 
	Rating:  4.5 
	Number of Reviews:  1000.0
- Out of Trouble™ 10 Minute Mask to Rescue Problem Skin 
	Rating:  4.5 
	Number of Reviews:  897.0
- Evercalm™ Gentle Cleansing Milk  
	Rating:  4.5 
	Number of Reviews:  363.0
- Vanilla Woods Eau de Parfum Rollerball 
	Rating:  4.5 
	Number of Reviews:  114.0
- Ros

Recommended products:
- Clear Improvement® Active Charcoal Mask to Clear Pores 
	Rating:  4.5 
	Number of Reviews:  4000.0
- Drink Up™ 10 Minute Hydrating Mask with Apricot & Swiss Glacier Water 
	Rating:  4.5 
	Number of Reviews:  306.0
- A Perfect World™ Age-Defense Treatment Lotion with White Tea 
	Rating:  4.5 
	Number of Reviews:  297.0
- A Perfect World™ Age-Defense Skin Guardian with White Tea 
	Rating:  4.5 
	Number of Reviews:  81.0
- GinZing™ Refreshing Eye Cream to Brighten and Depuff 
	Rating:  4.0 
	Number of Reviews:  4000.0

Product Name: WASO: Color-Smart Day Moisturizer SPF 30 Sunscreen
Rating: 4.0
Recommended products:
- WASO: Giga-Hydrating Rich Cream 
	Rating:  5.0 
	Number of Reviews:  121.0
- Urban Environment Oil-Free UV Protector Broad Spectrum Face Sunscreen SPF 42 
	Rating:  4.5 
	Number of Reviews:  1000.0
- Ultimate Sun Protector Lotion SPF 50+ Sunscreen 
	Rating:  4.5 
	Number of Reviews:  337.0
- WASO: Color-Smart Day Moisturizer Oil-Free SPF 30 Sunscreen 

Recommended products:
- Studio Skin Flawless 24 Hour Concealer 
	Rating:  4.5 
	Number of Reviews:  696.0
- Pep-Start Daily UV Protector Broad Spectrum SPF 50 
	Rating:  4.5 
	Number of Reviews:  286.0
- City Skin Age Defense Broad Spectrum SPF 50 PA++++ 
	Rating:  4.5 
	Number of Reviews:  236.0
- Broad Spectrum SPF 30 Mineral Sunscreen Lotion for Body 
	Rating:  4.5 
	Number of Reviews:  19.0
- Stay-Matte Oil-Free Makeup Foundation 
	Rating:  4.0 
	Number of Reviews:  1000.0

Product Name: Skin Tone Correcting & Beautifying BB Cream Sunscreen Broad Spectrum SPF 50
Rating: 3.5
Recommended products:
- Matte Loose Powder Mineral Foundation Broad Spectrum SPF 15 
	Rating:  4.5 
	Number of Reviews:  7000.0
- Pure Canvas Primer - Protecting Broad Spectrum SPF 30 
	Rating:  4.5 
	Number of Reviews:  6.0
- Stay Don’t Stray 360 Degree Stay Put Eyeshadow Primer  
	Rating:  4.0 
	Number of Reviews:  2000.0
- Bright Future Color Correctors  
	Rating:  4.0 
	Number of Reviews:  896.0
- Ultimate S